## Random Forest Model using Nevergrad gradient-free optimizer for hypertuning
This notebook aims to compare the performance of RF models using gridsearchcv hyperparameters tuning vs using Nevergrad. The experiment below is very "lite" and could only serve as a basis for further experimentation. This notebook is explained in a Toky Axel's Medium article : 

In [18]:
# Needed packages
#!pip install tqdm
#!pip install nevergrad

In [32]:
import tqdm
import pandas as pd
import nevergrad as ng
from scipy.io.arff import loadarff 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

### 1. Dataset sampling
More details about the dataset : https://www.openml.org/search?type=data&id=1494&sort=runs&status=active

In [43]:
raw = loadarff('qsar-biodeg.arff')
data = pd.DataFrame(raw[0])
data = data.replace({'Class': {b'2': 1, b'1': 0}})

In [44]:
X_train, X_test, y_train, y_test = train_test_split(data.drop("Class",axis=1), data.Class, test_size = 0.25, random_state = 123)

### 2. Model training using gridsearchcv hypertuning
More details about the model : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [45]:
y_train.shape

(791,)

In [46]:
tuned_parameters = {
    'n_estimators': [10, 100, 200], 
    'max_depth' : [None, 10, 20], 
    'min_samples_split' : range(2,50,20),
    'max_features' :['sqrt','log2', None]
}
clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5, scoring="roc_auc")
clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_split': range(2, 50, 20),
                         'n_estimators': [10, 100, 200]},
             scoring='roc_auc')

In [47]:
clf.best_params_

{'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_split': 2,
 'n_estimators': 200}

In [48]:
clf.best_score_ 

0.9254664053812455

In [49]:
best_clf = RandomForestClassifier(**clf.best_params_)
best_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=200)

#### Evaluate the model on Test set & Train set :  
The train AUC score below can lead us to overfitting

In [50]:
roc_auc_score(y_test, best_clf.predict_proba(X_test)[:,1])

0.9493178591271065

In [51]:
roc_auc_score(y_train, best_clf.predict_proba(X_train)[:,1])

0.9999491973176184

### 3. Model training using nevergrad
More details about the method : https://facebookresearch.github.io/nevergrad/machinelearning.html

We build a custom score that prevent us from overfitting : 
$$score = 1-auc\_on\_test + (w*abs((1-auc\_on\_train)-(auc\_on\_test)))$$
where $$ w = fixed\_weight\ (example\ 3) $$

In [62]:
def train_and_return_test_error(params, X_train, y_train, X_test, y_test):
    rf_clf = RandomForestClassifier(**params)
    rf_clf.fit(X_train,y_train)
    score_on_test = 1-roc_auc_score(y_test, rf_clf.predict_proba(X_test)[:,1])
    score_on_train = 1-roc_auc_score(y_train, rf_clf.predict_proba(X_train)[:,1])
    return [score_on_test, score_on_train]

# Parametrization is how nevergrad configures the optimizers.
# (https://facebookresearch.github.io/nevergrad/parametrization.html)
parametrization = ng.p.Dict(
    # Discrete params bounded by lower and upper bounds
    n_estimators = ng.p.Scalar(init=10, lower=2, upper=1000).set_integer_casting(),
    max_depth = ng.p.Scalar(init=None, lower=2, upper=1000).set_integer_casting(),
    min_samples_split = ng.p.Scalar(init=2, lower=2, upper=1000).set_integer_casting(),
    # Choice between given values.
    max_features=ng.p.Choice(['sqrt','log2', None]),
)

budget = 1000  # How many trainings we will do before concluding.

# List of some available optimizers
names = ["RandomSearch", "OnePlusOne", "CMA", "PSO", "ScrHammersleySearch"]

In [63]:
all_recommendation = {}

for name in names:
    optim = ng.optimizers.registry[name](parametrization=parametrization, budget=budget, num_workers=3)
    # Evaluate the model each 10 iterations
    for u in tqdm.tqdm(range(budget // 10)):
        # Ask and tell can be asynchronous.
        # Just be careful that you "tell" something that was asked.
        # Here we ask 3 times and tell 3 times in order to fake asynchronicity
        x1 = optim.ask()
        x2 = optim.ask()
        x3 = optim.ask()
        # The three following lines could be parallelized.
        # We could also do things asynchronously, i.e. do one more ask
        # as soon as a training is over.
        y1 = train_and_return_test_error(x1.value, X_train, y_train, X_test, y_test)
        y2 = train_and_return_test_error(x2.value, X_train, y_train, X_test, y_test)
        y3 = train_and_return_test_error(x3.value, X_train, y_train, X_test, y_test)
        # We want to minimize the test AUC 
        # but also the difference between train and test AUC
        # here we use Fixed Weight (3 times) for the difference
        y1_hat = y1[0]+(3*(abs(y1[1]-y1[0])))
        y2_hat = y2[0]+(3*(abs(y2[1]-y2[0])))
        y3_hat = y3[0]+(3*(abs(y3[1]-y3[0])))
        optim.tell(x1, y1_hat)
        optim.tell(x2, y2_hat)
        optim.tell(x3, y3_hat)
    recommendation = optim.recommend()
    score = train_and_return_test_error(recommendation.value, X_train, y_train, X_test, y_test)
    all_recommendation.update({name:recommendation.value})
    
    print(name, " provides a vector of parameters", 
          recommendation.value," with AUC on test :",
          1-score[0]," (AUC on train  ",1-score[1],")")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:56<00:00,  1.77s/it]


RandomSearch  provides a vector of parameters {'n_estimators': 17, 'max_depth': 582, 'min_samples_split': 106, 'max_features': 'sqrt'}  with AUC on test : 0.9149638866596704  (AUC on train   0.9260639533195925 )


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:10<00:00,  1.42it/s]


OnePlusOne  provides a vector of parameters {'n_estimators': 57, 'max_depth': 193, 'min_samples_split': 118, 'max_features': 'sqrt'}  with AUC on test : 0.9170319155503426  (AUC on train   0.9241407089151451 )


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:33<00:00,  1.53s/it]


CMA  provides a vector of parameters {'n_estimators': 65, 'max_depth': 311, 'min_samples_split': 104, 'max_features': None}  with AUC on test : 0.9196864003950861  (AUC on train   0.9330094057537666 )


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:33<00:00,  1.54s/it]


PSO  provides a vector of parameters {'n_estimators': 36, 'max_depth': 702, 'min_samples_split': 68, 'max_features': 'sqrt'}  with AUC on test : 0.9355515772578554  (AUC on train   0.9448536882747408 )


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:53<00:00,  2.93s/it]

ScrHammersleySearch  provides a vector of parameters {'n_estimators': 10, 'max_depth': 78, 'min_samples_split': 89, 'max_features': 'log2'}  with AUC on test : 0.9047163405148466  (AUC on train   0.920279705054141 )


In [64]:
all_recommendation

{'RandomSearch': {'n_estimators': 17,
  'max_depth': 582,
  'min_samples_split': 106,
  'max_features': 'sqrt'},
 'OnePlusOne': {'n_estimators': 57,
  'max_depth': 193,
  'min_samples_split': 118,
  'max_features': 'sqrt'},
 'CMA': {'n_estimators': 65,
  'max_depth': 311,
  'min_samples_split': 104,
  'max_features': None},
 'PSO': {'n_estimators': 36,
  'max_depth': 702,
  'min_samples_split': 68,
  'max_features': 'sqrt'},
 'ScrHammersleySearch': {'n_estimators': 10,
  'max_depth': 78,
  'min_samples_split': 89,
  'max_features': 'log2'}}